In [1]:
import sys

if str(type(sys.stdout)) == "<class 'ipykernel.iostream.OutStream'>":
    saved_std = sys.stdout
else:
    sys.stdout = saved_std

sys.stdout

### TO DO:
1. ~~Cleanup imports~~
1. ~~Decide on how to impute values for each column~~
1. ~~Use OHE and binarize the data.~~
1. ~~Make pipelines that process the data~~
1. ~~Use pipelines to fit models~~

In [2]:
#models:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier


#Preprocessing:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


#Other:
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('Data/modifiedData.csv')
test = pd.read_csv('Data/modifiedTest.csv')
y = df['Survived']
df.drop('Survived',axis=1,inplace=True)
display(df)
display(test)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,hasNChildren,hasCabin,has1or2sib,has3PSib
0,1,3,male,22.0,1,0,7.2500,S,Mr.,U,0,0,1,0
1,2,1,female,38.0,1,0,71.2833,C,Mrs.,C,0,1,1,0
2,3,3,female,26.0,0,0,7.9250,S,Miss.,U,0,0,0,0
3,4,1,female,35.0,1,0,53.1000,S,Mrs.,C,0,1,1,0
4,5,3,male,35.0,0,0,8.0500,S,Mr.,U,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,male,27.0,0,0,13.0000,S,Mr.,U,0,0,0,0
887,888,1,female,19.0,0,0,30.0000,S,Miss.,B,0,1,0,0
888,889,3,female,15.0,1,2,23.4500,S,Miss.,U,0,0,1,0
889,890,1,male,26.0,0,0,30.0000,C,Mr.,C,0,1,0,0


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,hasNChildren,hasCabin,has1or2sib,has3PSib
0,892,3,male,34.5,0,0,7.8292,Q,Mr.,U,0,0,0,0
1,893,3,female,47.0,1,0,7.0000,S,Mrs.,U,0,0,1,0
2,894,2,male,62.0,0,0,9.6875,Q,Mr.,U,0,0,0,0
3,895,3,male,27.0,0,0,8.6625,S,Mr.,U,0,0,0,0
4,896,3,female,22.0,1,1,12.2875,S,Mrs.,U,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,37.0,0,0,8.0500,S,Mr.,U,0,0,0,0
414,1306,1,female,39.0,0,0,108.9000,C,Mr.,C,0,1,0,0
415,1307,3,male,38.5,0,0,7.2500,S,Mr.,U,0,0,0,0
416,1308,3,male,26.0,0,0,8.0500,S,Mr.,U,0,0,0,0


In [ ]:
# Age & Fare will be filled in using the appropriate distribution (as normal doesn't fit).
# SibSp & Parch will be filled in as 0
# Pclass & Embarked will be filled in with modal class (3rd) 

In [ ]:
preSib = Pipeline(
    [('Sib_NA', SimpleImputer(strategy='constant', fill_value=0)),('Scaler',StandardScaler())])

preClass = Pipeline([('Class_NA', SimpleImputer(
    strategy='most_frequent')), ("OHE", OneHotEncoder(handle_unknown='ignore'))])

# handle_unknown='ignore' means that if the OHE encounters a class it doesn't recognize when transforming,
# it'll continue instead of throwing an error.

Preprocess = ColumnTransformer([('Scaler',StandardScaler(),
                                ['Age', 'Fare', 'hasNChildren', 'hasCabin', 'has1or2sib', 'has3PSib']),
                                ('FillNA', preSib, ['SibSp', 'Parch']), 
                                ('OHE',preClass,['Sex','Deck','Title','Embarked','Pclass'])])

The models I'm using are those suggested [here](https://www.kaggle.com/kenjee/titanic-project-example)

In [28]:
gnb_pipe = Pipeline([('Preprocess',Preprocess),('GNB',GaussianNB())])
cv = cross_val_score(gnb_pipe,df,y,cv=5)
print(cv.mean())

0.7677170296905406


In [29]:
tree_pipe = Pipeline([('Preprocess',Preprocess),('Tree',tree.DecisionTreeClassifier(random_state=7))])
cv = cross_val_score(tree_pipe,df,y,cv=5)
print(cv.mean())

0.7620802209528592


In [33]:
SVC_pipe = Pipeline([('Preprocess',Preprocess),('SVC',SVC(probability = True))])
cv = cross_val_score(SVC_pipe,df,y,cv=5)
print(cv.mean()) # Maybe try one with less overlapping features.

0.827154604230745


In [37]:
xgb_pipe = Pipeline([('Preprocess',Preprocess),
                     ('xgb',XGBClassifier(random_state=7,use_label_encoder=False,eval_metric='logloss'))])
cv = cross_val_score(xgb_pipe,df,y,cv=5)
print(cv.mean()) # Maybe try one with less overlapping features.

0.8114744837110036


In [38]:
RF_pipe = Pipeline([('Preprocess',Preprocess),('RF',RandomForestClassifier(random_state=7))])
cv = cross_val_score(RF_pipe,df,y,cv=5)
print(cv.mean())

0.8024731655263324


In [39]:
KNN_pipe = Pipeline([('Preprocess',Preprocess),('KNN',KNeighborsClassifier())])
cv = cross_val_score(KNN_pipe,df,y,cv=5)
print(cv.mean())

0.7957504237022157


In [40]:
logreg_pipe = Pipeline([('Preprocess',Preprocess),('Logreg',LogisticRegression())])
cv = cross_val_score(logreg_pipe,df,y,cv=5)
print(cv.mean())

0.8249325214989642


In [41]:
voting_clf = VotingClassifier(estimators = [('lr',logreg_pipe),('knn',KNN_pipe),('rf',RF_pipe),
                                            ('svc',SVC_pipe),('xgb',xgb_pipe)], voting = 'soft')
cv = cross_val_score(voting_clf,df,y,cv=5)
print(cv.mean())

0.8327851358985626


In [47]:
voting_clf.fit(df,y)
data = np.array([test.index, voting_clf.predict(test)]).T
noTunePred = pd.DataFrame(data = data, columns = ['PassengerId','Survived'])
noTunePred.to_csv('Predictions/basePrediction.csv', index=False)

Now tuning:

In [ ]:
#simple performance reporting function
def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [76]:
logreg_pipe = Pipeline([('Preprocess',Preprocess),('Logreg',LogisticRegression())])
param_grid = {'Logreg__max_iter' : [2000],
              'Logreg__penalty' : ['l1', 'l2'],
              'Logreg__C' : np.logspace(-4, 4, 20),
              'Logreg__solver' : ['liblinear']}

lr_grid = GridSearchCV(logreg_pipe, param_grid=param_grid, verbose = 1, n_jobs = -1)
best_lr = lr_grid.fit(df,y)
print('Done with grid :)\n')

clf_performance(best_lr,'Logistic Regression')

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Done with grid :)

Logistic Regression
Best Score: 0.8282907538760906
Best Parameters: {'Logreg__C': 0.615848211066026, 'Logreg__max_iter': 2000, 'Logreg__penalty': 'l2', 'Logreg__solver': 'liblinear'}


In [77]:
knn_pipe = Pipeline([('Preprocess',Preprocess),('KNN',KNeighborsClassifier())])

param_grid = {'KNN__n_neighbors' : [3,5,7,9],
              'KNN__weights' : ['uniform', 'distance'],
              'KNN__algorithm' : ['auto', 'ball_tree','kd_tree'],
              'KNN__p' : [1,2]}

knn_grid = GridSearchCV(knn_pipe, param_grid=param_grid, verbose = 1, n_jobs = -1)
best_knn = knn_grid.fit(df,y)
print('Done with grid :)\n')

clf_performance(best_knn,'KNN')

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Done with grid :)

KNN
Best Score: 0.8226853304877283
Best Parameters: {'KNN__algorithm': 'auto', 'KNN__n_neighbors': 7, 'KNN__p': 1, 'KNN__weights': 'uniform'}


In [9]:
SVC_pipe = Pipeline([('Preprocess',Preprocess),('SVC',SVC(probability = True))])

param_grid = tuned_parameters = [{'SVC__kernel': ['rbf'], 'SVC__gamma': [.1,.5,1,2,5,10],
                                  'SVC__C': [.1, 1, 10, 100]},
                                 {'SVC__kernel': ['linear'], 'SVC__C': [.1, 1, 10, 100]},
                                 {'SVC__kernel': ['poly'], 'SVC__degree' : [2,3,4,5], 'SVC__C': [.1, 1, 10, 100]}]

sys.stdout = saved_std
SVC_grid = GridSearchCV(SVC_pipe, param_grid=param_grid, verbose = 1)
best_SVC = SVC_grid.fit(df,y)
print('Done with grid :)\n')


clf_performance(best_SVC,'SVC')

Fitting 5 folds for each of 44 candidates, totalling 220 fits
Done with grid :)

SVC
Best Score: 0.8293892411022534
Best Parameters: {'SVC__C': 1, 'SVC__degree': 2, 'SVC__kernel': 'poly'}


In [14]:
RF_pipe = Pipeline([('Preprocess', Preprocess),
                   ('RF', RandomForestClassifier(random_state=7))])

param_grid = {'RF__n_estimators': [400, 450, 500, 550],
              'RF__criterion': ['gini', 'entropy'],
              'RF__bootstrap': [True],
              'RF__max_depth': [15, 20, 25],
              'RF__max_features': ['auto', 'sqrt', 10],
              'RF__min_samples_leaf': [2, 3],
              'RF__min_samples_split': [2, 3]}

RF_rand = RandomizedSearchCV(
    RF_pipe, param_distributions=param_grid, n_iter=100, cv=5, verbose=2, n_jobs=-1)
best_RF = RF_rand.fit(df, y)
clf_performance(best_RF, 'Random Forest')

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=20, RF__max_features=10, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=550; total time=   1.5s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=sqrt, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=500; total time=   1.3s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=20, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=450; total time=   1.2s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=auto, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=400; total time=   1.0s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=550; total time=   2.0s
[CV] END RF__bootstrap=True, RF__crite

[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=450; total time=   1.5s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=sqrt, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=450; total time=   1.7s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=20, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=550; total time=   2.4s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=450; total time=   1.7s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=auto, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=450; total time=   1.8s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=10, RF__m

[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=25, RF__max_features=10, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=400; total time=   1.7s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=500; total time=   2.1s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=auto, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=550; total time=   2.2s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=25, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=500; total time=   2.0s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=auto, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=550; total time=   2.3s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=auto, RF__m

[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=500; total time=   2.5s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=25, RF__max_features=auto, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=500; total time=   2.1s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=25, RF__max_features=auto, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=550; total time=   2.2s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=auto, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=400; total time=   1.8s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=sqrt, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=550; total time=   2.1s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=10, RF__min_

[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=500; total time=   1.8s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=550; total time=   2.3s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=25, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=550; total time=   2.2s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=sqrt, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=400; total time=   1.6s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=20, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=400; total time=   1.7s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=10, RF__min_sa

[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=20, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=400; total time=   1.9s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=10, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=450; total time=   2.3s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=400; total time=   1.7s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=auto, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=500; total time=   1.9s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=10, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=550; total time=   2.3s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=10, RF_

[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=sqrt, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=450; total time=   1.7s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=auto, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=500; total time=   1.8s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=450; total time=   1.7s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=10, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=450; total time=   2.0s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=2, RF__n_estimators=500; total time=   1.8s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=20, RF__max_features=10, RF

[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=20, RF__max_features=auto, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=500; total time=   2.0s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=25, RF__max_features=10, RF__min_samples_leaf=3, RF__min_samples_split=3, RF__n_estimators=550; total time=   2.3s
[CV] END RF__bootstrap=True, RF__criterion=gini, RF__max_depth=15, RF__max_features=sqrt, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=550; total time=   2.1s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=15, RF__max_features=auto, RF__min_samples_leaf=2, RF__min_samples_split=3, RF__n_estimators=500; total time=   1.9s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=sqrt, RF__min_samples_leaf=3, RF__min_samples_split=2, RF__n_estimators=500; total time=   1.9s
[CV] END RF__bootstrap=True, RF__criterion=entropy, RF__max_depth=20, RF__max_features=10, RF